# Forth Problem

## Imports

In [1]:
import sys
sys.path.append("../../")
sys.path.append("../")
from dlpmln import DeepLPMLN
import torch

import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import time

from data_function import create_data_sample,format_dataList,format_observations,add_test,carry_test

## dprogram

Specify the DeepLPMLN program.

In [2]:
dprogram='''
num1(1,num1_1).
num1(0,num1_0).
num2(1,num2_1).
num2(0,num2_0).
1{carry(0,Carry): Carry=0..1}1.
nn(m1(A, B, Carry, 1), result, [0,1,2,3,4,5,6,7,8,9]) :- num1(P,A), num2(P,B), Carry=0..1.
nn(m2(A, B, Carry, 1), carry, [0,1]) :- num1(P,A), num2(P,B), Carry=0..1.

result(P,X) :- num1(P, A), num2(P, B), carry(P, Carry), result(A,B,Carry,0,X).
carry(P+1,X) :- num1(P, A), num2(P, B), carry(P, Carry), carry(A,B,Carry,0,X).
'''

## Neural Network Class

Create the neural network class that will be used for our data.

In [3]:
class FC(nn.Module):

    def __init__(self, *sizes):
        super(FC, self).__init__()
        layers = []
        for i in range(len(sizes)-2):
            layers.append(nn.Linear(sizes[i], sizes[i+1]))
            layers.append(nn.ReLU())
        layers.append(nn.Linear(sizes[-2], sizes[-1]))
        layers.append(nn.Softmax(1))
        self.nn = nn.Sequential(*layers)

    def forward(self, x):
        return self.nn(x)

## Neural Network Instantiation
- Instantiate neural networks.
- Map neural networks into the NNMapping function called 'function'.
- Specify the optimizers for each network (we use the Adam optimizer here).

In [4]:
m1 = FC(30,25,10) # network for adding the numbers
m2 = FC(30,5,2)   # network for finding the carry out


functions = {'m1':m1, 'm2':m2} #NNMapping

optimizers = {'m1':torch.optim.Adam(m1.parameters(), lr=0.01),'m2':torch.optim.Adam(m2.parameters(), lr=0.01)}


Below are 20 samples of data.

In [5]:
for i in range(20):
    print(create_data_sample()[0])

add([6,2],[6,2],0,[1,2,4]).
add([8,9],[3,0],0,[1,1,9]).
add([8,8],[7,5],1,[1,6,4]).
add([3,6],[7,0],1,[1,0,7]).
add([0,3],[8,8],1,[0,9,2]).
add([0,3],[3,3],0,[0,3,6]).
add([5,3],[1,3],1,[0,6,7]).
add([6,4],[8,5],1,[1,5,0]).
add([0,8],[2,6],0,[0,3,4]).
add([6,9],[7,6],1,[1,4,6]).
add([6,2],[3,7],1,[1,0,0]).
add([7,2],[9,0],1,[1,6,3]).
add([9,8],[7,4],0,[1,7,2]).
add([5,8],[4,1],1,[1,0,0]).
add([3,1],[6,1],1,[0,9,3]).
add([3,6],[9,8],0,[1,3,4]).
add([8,1],[7,1],1,[1,5,3]).
add([2,8],[0,4],0,[0,3,2]).
add([8,4],[5,7],0,[1,4,1]).
add([6,2],[8,6],1,[1,4,9]).


In [6]:
print('\n\n')

obs,str_list=create_data_sample() #data sample the same as DeepProbLog used 
print(obs+'\n\n')

dl=format_dataList(obs,str_list) # dataList item
print(dl)

o=format_observations(obs,str_list) # obsList item
print(o)




add([5,4],[8,9],0,[1,4,3]).


{'num1_0,num2_0,0': tensor([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]), 'num1_0,num2_0,1': tensor([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]]), 'num1_1,num2_1,0': tensor([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]), 'num1_1,num2_1,1': tensor([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]])}

    :-not carry(0,0).
    :-not carry(2,1).
    :-not result(1,4).
    :-not result(0,3).
    


## Create dataList and obsList 

This the code that will use format_dataList() and format_observations() to build the data set that our DeepLPMLN object will train on.

In [7]:
dataList = []
obsList = []

train_size=512
test_size=512


for i in range(train_size):
    
    obs,str_list=create_data_sample()
    
    dataList.append(format_dataList(obs,str_list))
    obsList.append(format_observations(obs,str_list))

## Create test dataset and dataLoader for neural network "m1"

In [8]:
add_test_dataset=add_test(test_size)

add_test_dataloader=DataLoader(add_test_dataset,batch_size=4,shuffle=True)

carry_test_dataset=carry_test(test_size)

carry_test_dataloader=DataLoader(carry_test_dataset,batch_size=4,shuffle=True)



## Create DeepLPMLN object

In [9]:
dlpmlnObj = DeepLPMLN(dprogram, functions, optimizers, dynamicMVPP=False)
#dlpmlnObj.device='cpu' #put the training on the CPU

## Training and Testing 

In [10]:
print('training...')
test_acc1=[]
test_acc2=[]

for i in range(40):
    time1 = time.time()
    dlpmlnObj.learn(dataList=dataList, obsList=obsList, epoch=1)
    time2 = time.time()
    test_acc1.append(dlpmlnObj.testNN("m1", add_test_dataloader)) #test m1 network
    test_acc2.append(dlpmlnObj.testNN("m2", carry_test_dataloader)) #test m2 network
    print("--- train time: %s seconds ---" % (time2 - time1))
    print("--- test time: %s seconds ---" % (time.time() - time2))

training...
Training for epoch 1 ...
Test Accuracy on NN Only for m1: 27%
Test Accuracy on NN Only for m2: 95%
--- train time: 67.33205151557922 seconds ---
--- test time: 0.31531667709350586 seconds ---
Training for epoch 1 ...
Test Accuracy on NN Only for m1: 61%
Test Accuracy on NN Only for m2: 97%
--- train time: 68.79846572875977 seconds ---
--- test time: 0.3114442825317383 seconds ---
Training for epoch 1 ...
Test Accuracy on NN Only for m1: 77%
Test Accuracy on NN Only for m2: 98%
--- train time: 69.78857159614563 seconds ---
--- test time: 0.32141780853271484 seconds ---
Training for epoch 1 ...
Test Accuracy on NN Only for m1: 87%
Test Accuracy on NN Only for m2: 99%
--- train time: 69.18318724632263 seconds ---
--- test time: 0.35251712799072266 seconds ---
Training for epoch 1 ...
Test Accuracy on NN Only for m1: 92%
Test Accuracy on NN Only for m2: 99%
--- train time: 67.57443952560425 seconds ---
--- test time: 0.13956046104431152 seconds ---
Training for epoch 1 ...
Test